A while back I wrote code to perform power flow for a particular instanton candidate. Unfortunately, I did not have the forethought to add an interface to this code, so now I need to go back and study it to know how to use it.

Every function that the user interacts with should accept an instance of either `InstantonInputData` or `InstantonOutputData`.

In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton
d = testcase("timing")
o = solve_temporal_instanton(d)

In [15]:
import TemporalInstanton.return_conductor_params
import TemporalInstanton.return_thermal_constants
idx = 1
T = round(Int64,length(d.D0)/length(d.k))
conductor_name = d.line_conductors[idx]
conductor_params = return_conductor_params(conductor_name)
line = d.lines[idx]
line_params = LineParams(line[1],line[2],d.res[idx],d.reac[idx],d.line_lengths[idx])

(t_a,t_c,t_d,t_f) = return_thermal_constants(line_params,conductor_params
,d.Tamb,d.Sb,d.int_length,T,d.T0)



(-0.003270204441451697,275.9154973489844,0.15284183206941251,46.77452937287844)

`o.x` contains an element for each instanton candidate. Within each element, there is a set of $k$ vectors, where $k$ is the number of time steps. Each of these $k$ vectors contains wind deviations for the nodes in `Ridx`, so it has as many entries as there are wind nodes.

We can solve a DC power flow at each time step by doing the following:

1. Add the wind deviations at the time step to the dispatch vector.
2. Add all injections to get $P$.
3. Use a simple function to generate $B$ (which includes admittance matrix and droop response).
4. Factorize $B$ to obtain $\theta=B^{-1}P$.

Now that we have the set of voltage angles $\theta$ at the given time step, we can do two things:

1. Compute the power flow on each line by taking $P_{ij}=B_{ij}(\theta_i-\theta_j)$.
2. Compute instantaneous losses on each line (function of angle difference). When this is done for all time steps, we can then use the heat balance IVP to obtain temperature trajectories for all lines.

In [37]:
o.x[1][3]

18-element Array{Float64,1}:
 -0.00751789 
 -4.49168e-6 
  0.000181627
  0.000207159
  0.00021905 
  0.000103256
  3.5107e-5  
  0.000230093
 -2.3748e-6  
 -5.75214e-5 
 -3.78247e-5 
 -3.51509e-5 
 -3.1528e-5  
 -2.19202e-5 
 -1.6636e-5  
  0.000113861
  0.000116112
  0.000115214

In [64]:
"""
    angles,mismatch = powerflow(d,o,eIdx)
Solve DC power flow for temporal instanton input 
data `d` and the instanton candidate with index
`eIdx` of output data `o`.
"""
function powerflow(
    d::InstantonInputData,
    o::InstantonOutputData,
    eIdx::Int64
    )
    T = round(Int64,length(d.D0)/length(d.k))
    Y = d.Y
    ref = d.ref
    k = d.k
    n = length(k)
    G0 = d.G0
    D0 = d.D0
    Ridx = d.Ridx
    R0 = d.R0
    x = o.x
    
    wind_dev = Array(Float64,0)
    for i in 1:T
        append!(wind_dev,x[eIdx][i])
    end

    # fixed wrt nodal injections:
    B = fixed_wind_A(T,Y,ref,k)
    
    # put wind deviation elements in
    # correct positions:
    wind_dev = expand_renewable_vector(wind_dev,Ridx,n,T)

    # net renewable:
    Rnet = wind_dev + R0

    # compute P:
    P = fixed_wind_b(n,T,G0,Rnet,D0)
    theta = B\P
    angles,alpha = return_angles(theta,n,T)
    return angles,alpha
end

powerflow (generic function with 1 method)

In [71]:
angles,alpha = powerflow(d,o,1)

angles[1]

73-element Array{Float64,1}:
  3.83668e-16
  0.00106659 
 -0.0515348  
  0.0114302  
  0.0186659  
  0.0383353  
  0.0675333  
  0.0673786  
 -0.0253425  
  0.0011672  
 -0.0913642  
 -0.103502   
 -0.134472   
  ⋮          
 -0.0491909  
 -0.159757   
 -0.156796   
 -0.197069   
 -0.208344   
 -0.159506   
 -0.183288   
 -0.21803    
 -0.275148   
 -0.204653   
 -0.09034    
 -0.21444    

In [ ]:
 
        
conductor_name = d.line_conductors[idx]
conductor_params = return_conductor_params(conductor_name)
line = d.lines[idx]
line_params = LineParams(line[1],line[2],d.res[idx],d.reac[idx],d.line_lengths[idx])

(t_a,t_c,t_d,t_f) = return_thermal_constants(line_params,conductor_params
,d.Tamb,d.Sb,d.int_length,T,d.T0)